# Merging staff.am and hr.am data

In order to keep data consistency, it was decided to concatenate separate datasets one by one.
In the following notebook, staff.am and hr.am datasets were combined, using several principles:

     -keep features common for the most websites,
     -drop features having large number of missing values and unique for current dataset,
     -merging features having similar/same meanings(i.e. Prof Skills and Soft Skills are merged under "Skills")
   

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import datetime
import pymongo
from pymongo import MongoClient
import json
from itertools import chain
import numpy as np
from fuzzywuzzy import fuzz
import ast


C:\Users\Srbuhi\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
def read_from_mongoDB(database, collection): 
    client = MongoClient('localhost', 27017)
    db = client[database]
    coll = db[collection]
    coll_df = pd.DataFrame(list(coll.find({})))
    return coll_df

In [3]:
full=read_from_mongoDB("labor_market","full_am")
hr_am=read_from_mongoDB("labor_market","hr_am")
ijob_am=read_from_mongoDB("labor_market","ijob_am")
job_am=read_from_mongoDB("labor_market","job_am")
myjob_am=read_from_mongoDB("labor_market","myjob")
staff_am=read_from_mongoDB("labor_market","staff_am")
jobfinder=read_from_mongoDB("labor_market","jobfinder_am")
career_center2=read_from_mongoDB("labor_market","career_center2")
worknet=read_from_mongoDB("labor_market","worknet_am")

# Staff.am

The following features were dropped:
 - "Website","Active Jobs","Job Description","Job History", "Prof Skills","Soft Skills", "Page Views","Qualification"
 
      Website had no observations,
      "Job Description"'s info is mostly contained in Job Content,thus the former was dropped.
      "Page Views" contained info on total number of views of company job posting, which is not 
      relevant and not available for other websites, so it was dropped. 
      "Active jobs" is number of current postings of a company, which can be derived from scraped
      data if needed.
      "Job History"-not present elsewhere, so was dropped.
      Soft and Prof skills were merged into one column and the separate features were dropped.
      "Qualification" is 99.94% is contained in "Job Content", as revealed by FuzzyWuzzy ratio.
 -  "Job Category" was renamed to "Employment Type","Job Type" to "Job Industry","Industry"
     to "Company Industry" to match actual meanings

In [4]:
staff_am.columns

Index(['Active Jobs', 'Company', 'Deadline', 'Employment Term', 'Industry',
       'Job Category', 'Job Content', 'Job Description', 'Job History',
       'Job Link', 'Job Location', 'Job Title', 'Job Type', 'Job Views',
       'Page Views', 'Prof Skills', 'Qualification', 'Salary', 'Scrape_Date',
       'Soft Skills', 'Website', '_id'],
      dtype='object')

In [5]:
#Job Description and Job Content are in fact same thing,
#manual exlporation showed that Job Description is just the first sentence of Job Content.
staff_am[["Job Description","Job Content"]][staff_am["Job Description"]!="['Issue']"].sample(5)

,Job Description,Job Content
2113,We are looking for a Receptionist to manage ou...,['\nWe are looking for a Receptionist to manag...
4385,all.me is an international IT company with hea...,[\nall.me is an international IT company with ...
4447,None,"[\nՀՀ կրթության, գիտության, մշակույթի և սպորտի..."
2109,At DISQO we believe the best software is writt...,['\nAt DISQO we believe the best software is w...
2235,Միջազգային բեռնափոխադրում իրականացնող ընկերութ...,['\nՄիջազգային բեռնափոխադրում իրականացնող ընկե...


In [6]:
#Website has no non-missing value
website=(staff_am["Website"].isna().sum()+(staff_am["Website"]=="").sum())/len(staff_am)*100
print(f"Percentage of missing values in 'Website'column:{website.round(2)}%",)

Percentage of missing values in 'Website'column:100.0%


In [7]:
#getting rid of exceptions scraped as "['Issue']"
for index, row in staff_am.iterrows():
    for i in staff_am.columns:
        if row[i]=="['Issue']":
            row[i]=""

In [8]:
#Merging Prof Skills and Soft Skills into one
staff_am["Prof Skills"]=staff_am["Prof Skills"].apply(lambda x:str(x).replace("[","").replace("]",""))
staff_am["Soft Skills"]=staff_am["Soft Skills"].apply(lambda x:str(x).replace("[","").replace("]",""))
staff_am["Skills"]=staff_am.loc[:,["Prof Skills","Soft Skills"]].sum(axis=1)

In [9]:
#Renaming features to express the exact meaning presented on the website
staff_am.rename(columns={"Job Category":"Employment Type","Job Type":"Job Industry","Industry":"Company Industry"}, inplace=True)

In [10]:
#making strings
staff_am["Job Content"]=staff_am["Job Content"].fillna("").apply(lambda x:"".join(x).replace("[","").replace("]",""))
staff_am["Qualification"]=staff_am["Qualification"].fillna("").apply(lambda x:"".join(x).replace("[","").replace("]",""))
staff_am["Salary"]=staff_am["Salary"].fillna("").apply(lambda x:"".join(x).replace("[","").replace("]",""))

In [11]:
#making full link
staff_am["Job Link"]=staff_am["Job Link"].apply(lambda x:"https://staff.am"+x)

In [12]:
#filling empty strings with NaNs and then replacing them with None
staff_am=staff_am.replace(r'^\s*$', np.nan, regex=True)

In [13]:
staff_am=staff_am.where(staff_am.notna(),None)

In [14]:
#Let's see what percentage of Non-Null Qualification is contained in Job Content
ans=0
for index,row in staff_am.iterrows():
    if row["Qualification"] is not None or "" and row["Job Content"] is not None or "":
        Ratio = fuzz.ratio(row["Qualification"].lower(),row["Job Content"].lower())
        if Ratio==0:
            print(row[["Qualification","Job Content"]])
            print("______________________________________________")
            ans+=1
print("Percentage of Qualifications Not contained in Job Content:",(ans/len(staff_am)*100))

Qualification    'BD Expert Evaluation', 'Analysis and appraisa...
Job Content      '\nall.me is an international IT company with ...
Name: 141, dtype: object
______________________________________________
Qualification    'BD Expert Evaluation', 'Analysis and appraisa...
Job Content      '\nall.me is an international IT company with ...
Name: 1054, dtype: object
______________________________________________
Qualification    'Data collection,', 'Data analysis,', 'Support...
Job Content      '\nWe are looking for an intern, an energetic,...
Name: 2936, dtype: object
______________________________________________
Percentage of Qualifications Not contained in Job Content: 0.061703002879473466


In [15]:
#Dropping the noted features
staff_am.drop(["Website","Active Jobs","Job Description","Job History", "Prof Skills","Soft Skills", "Page Views", "Qualification"], axis=1, inplace=True)

In [16]:
staff_am.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4862 entries, 0 to 4861
Data columns (total 15 columns):
Company             4849 non-null object
Deadline            4849 non-null object
Employment Term     4849 non-null object
Company Industry    4542 non-null object
Employment Type     4849 non-null object
Job Content         4849 non-null object
Job Link            4862 non-null object
Job Location        4835 non-null object
Job Title           4849 non-null object
Job Industry        4849 non-null object
Job Views           4849 non-null object
Salary              3628 non-null object
Scrape_Date         603 non-null object
_id                 4862 non-null object
Skills              4796 non-null object
dtypes: object(15)
memory usage: 569.8+ KB


In [17]:
staff_am.head()

,Company,Deadline,Employment Term,Company Industry,Employment Type,Job Content,Job Link,Job Location,Job Title,Job Industry,Job Views,Salary,Scrape_Date,_id,Skills
0,Mamble,27 July 2019,Permanent,Information technologies,Full time,'\nDESCRIPTION\nMamble seeks for a strong Fron...,https://staff.am/en/front-end-developer-112,Yerevan,Front-end Developer,Software development,100,None,None,5d15241ac8d3f9226842a407,"'JavaScript Frameworks', 'JavaScript'"
1,PicsArt,27 July 2019,Permanent,Information technologies,Full time,"'\nAs a QA Engineer at PicsArt, you’ll ensure ...",https://staff.am/en/qa-engineer-175,Yerevan,QA Engineer,Quality Assurance /Control,42,None,None,5d15241ac8d3f9226842a408,"'Java', 'Python', 'QA', 'JavaScript'"
2,PicsArt,27 July 2019,Permanent,Information technologies,Full time,'\nPicsArt\xa0is actively seeking for a talent...,https://staff.am/en/java-engineer,Yerevan,Java Engineer,Software development,65,None,None,5d15241ac8d3f9226842a409,"'Python', 'Big Data'"
3,Digitain,27 July 2019,Permanent,Information technologies,Full time,"""\nDigitain is looking for a Procurement Speci...",https://staff.am/en/procurement-specialist-1,Yerevan,Procurement Specialist,Procurement/Logistics/Courier,74,email protected,None,5d15241ac8d3f9226842a40a,"'Flexible', 'Detail-oriented', 'Multitasking'"
4,PicsArt,27 July 2019,Permanent,Information technologies,Full time,"'\nAs a DevOps Engineer at PicsArt, you’ll hav...",https://staff.am/en/devops-engineer-63,Yerevan,DevOps Engineer,Software development,33,None,None,5d15241ac8d3f9226842a40b,"'Python', 'DevOps', 'Big Data'"


# HR.am

 The following features were dropped:
  - "Applications", "Benefits", "Education","Languages","Job Type","Prof Skills","Soft Skills"
    The first four either had many missing values or features that were not contained in other datasets.
      "Job Type" has the same values as "Job Title", so it was dropped not to have duplicated columns.
      "Prof Skills" and "Soft Skills"
  - The following features were renamed:Job Description":"Job Content","Qualification":"Job Industry","Page Views":"Job Views",
     to match actual meanings
    

In [18]:
apps=(hr_am["Applications"].isna().sum()+(hr_am["Applications"]==0).sum()+(hr_am["Applications"]=="0").sum())/len(hr_am)*100
print(f"Percentage of missing values and 0s in 'Applications column':{apps.round(2)}%",)
benefs=(hr_am["Benefits"].isna().sum()+(hr_am["Benefits"]=="[]").sum())/len(hr_am)*100
print(f"Percentage of missing values in 'Benefits'column:{benefs.round(2)}%",)
print("Percentage of missing values in Education",((hr_am.astype(str)['Education'] == '[]').sum()/len(hr_am)*100).round(2),"%")
print("Percentage of missing values in Languages:",(((hr_am["Languages"]=="[]").sum()+hr_am["Languages"].isna().sum())/len(hr_am)*100).round(2),"%")
print("Percentage of same values in Job Title and Job Type:",(((hr_am["Job Title"]==hr_am["Job Type"]).sum()/len(hr_am))*100).round(2),"%")

Percentage of missing values and 0s in 'Applications column':84.21%
Percentage of missing values in 'Benefits'column:73.29%
Percentage of missing values in Education 75.35 %
Percentage of missing values in Languages: 60.58 %
Percentage of same values in Job Title and Job Type: 99.93 %


In [19]:
#merging Prof and Soft skills into Skills
hr_am["Prof Skills"]=hr_am["Prof Skills"].apply(lambda x:str(x).replace("[","").replace("]",""))
hr_am["Soft Skills"]=hr_am["Soft Skills"].apply(lambda x:str(x).replace("[","").replace("]",""))
hr_am["Skills"]=hr_am.loc[:,["Prof Skills","Soft Skills"]].sum(axis=1)

In [20]:
hr_am.drop(["Applications","Benefits", "Job Type","Languages","Education","Prof Skills","Soft Skills"],axis=1,inplace=True)

In [21]:
#renaming respective columns to match actual meaning
hr_am.rename(columns={ "Job Description":"Job Content","Qualification":"Job Industry","Page Views":"Job Views"},inplace=True)

In [22]:
#function keeping only 1st element of 0rd tuple
def clean_str():
    ans = []

    try:
        for index,row in hr_am.iterrows():
            if row['Job Industry']:
                literal = row['Job Industry']
                if type(row['Job Industry']) is str:
                    literal = ast.literal_eval(literal)
                if not literal:
                    ans.append(None)
                    continue
                                    
                clean_st = " ".join(literal[0][1])
                ans.append(clean_st)
            else:
                ans.append(None)
            
    except RuntimeError as e:
        print(e)
        return ans
    return ans

In [23]:
clean=clean_str()

In [24]:
hr_am["Job Industry"]=clean

In [25]:
#function transforming list of lists and tuples into list
#flatten = lambda *n: (e for a in n
#    for e in (flatten(*a) if isinstance(a, (tuple, list)) else (a,)))

In [26]:
#getting rid of list of lists, tuples and empty lists
#hr_am["Job Industry"]=hr_am["Job Industry"].apply(lambda x:list(flatten(x)))
#hr_am["Job Industry"]=hr_am["Job Industry"].apply(lambda x: "".join(x).replace("[","").replace("]","").replace("(","").replace(")",""))
#hr_am["Job Industry"].apply(lambda x:x.split(",")[:-1][1:]).value_counts()

In [27]:
hr_am.head()

,Company,Deadline,Job Content,Job Link,Job Title,Job Views,Job Industry,Scrape_Date,_id,Skills
0,"Fora LLC - Երևան, ՀՀ","24 August, 2019",Աշխատանքի նկարագիր. \rՍիքսթ ավտովարձույթի գործ...,http://hr.am/vacancy/view/vid/69316/t/,Ավտովարձույթի գործակալ,172,None,NaN,5d5670460cc4ac20014591fe,
1,"Decora-Group LLC - 88/8 Araratyan St., yerevan","15 August, 2019",JOB DESCRIPTION:\r•\t Under the supervision of...,http://hr.am/vacancy/view/vid/69315/t/,FINANCIAL SPECIALIST,134,None,NaN,5d5670460cc4ac20014591ff,'Analytical thinking '
2,"ԱՎՏՈԲԱՏ ՍՊԸ - Հայաստան, Երևան","24 August, 2019",Պարտականությունները՝\r• որոնել նոր արտասահմանյ...,http://hr.am/vacancy/view/vid/69311/t/,"Ներմուծման, արտաքին և ներքին կապերի մենեջեր",None,None,NaN,5d5670460cc4ac2001459200,"('MS Office', '3\xa0years')'Ability to priorit..."
3,Հայկական Ծրագրեր ՍՊԸ (Armenian Software) - Երևան,"24 August, 2019",Հայկական Ծրագրեր ընկերությունը փնտրում է Համակ...,http://hr.am/vacancy/view/vid/69310/t/,Համակարգային ադմինիստրատոր,None,None,NaN,5d5670460cc4ac2001459201,
4,Scopic Software - Remote,"21 August, 2019",Scopic Software is seeking an experienced Mark...,http://hr.am/vacancy/view/vid/69309/t/,Marketing Specialist / Content Writer,None,None,NaN,5d5670460cc4ac2001459202,


In [28]:
hr_am=hr_am.replace(r'^\s*$', np.nan, regex=True)
hr_am=hr_am.where(hr_am.notna(),None)

In [30]:
hr_am.head()

,Company,Deadline,Job Content,Job Link,Job Title,Job Views,Job Industry,Scrape_Date,_id,Skills
0,"Fora LLC - Երևան, ՀՀ","24 August, 2019",Աշխատանքի նկարագիր. \rՍիքսթ ավտովարձույթի գործ...,http://hr.am/vacancy/view/vid/69316/t/,Ավտովարձույթի գործակալ,172,None,None,5d5670460cc4ac20014591fe,None
1,"Decora-Group LLC - 88/8 Araratyan St., yerevan","15 August, 2019",JOB DESCRIPTION:\r•\t Under the supervision of...,http://hr.am/vacancy/view/vid/69315/t/,FINANCIAL SPECIALIST,134,None,None,5d5670460cc4ac20014591ff,'Analytical thinking '
2,"ԱՎՏՈԲԱՏ ՍՊԸ - Հայաստան, Երևան","24 August, 2019",Պարտականությունները՝\r• որոնել նոր արտասահմանյ...,http://hr.am/vacancy/view/vid/69311/t/,"Ներմուծման, արտաքին և ներքին կապերի մենեջեր",None,None,None,5d5670460cc4ac2001459200,"('MS Office', '3\xa0years')'Ability to priorit..."
3,Հայկական Ծրագրեր ՍՊԸ (Armenian Software) - Երևան,"24 August, 2019",Հայկական Ծրագրեր ընկերությունը փնտրում է Համակ...,http://hr.am/vacancy/view/vid/69310/t/,Համակարգային ադմինիստրատոր,None,None,None,5d5670460cc4ac2001459201,None
4,Scopic Software - Remote,"21 August, 2019",Scopic Software is seeking an experienced Mark...,http://hr.am/vacancy/view/vid/69309/t/,Marketing Specialist / Content Writer,None,None,None,5d5670460cc4ac2001459202,None


In [32]:
merged=pd.concat([staff_am,hr_am], ignore_index=True, sort=True)

In [33]:
merged.head()

,Company,Company Industry,Deadline,Employment Term,Employment Type,Job Content,Job Industry,Job Link,Job Location,Job Title,Job Views,Salary,Scrape_Date,Skills,_id
0,Mamble,Information technologies,27 July 2019,Permanent,Full time,'\nDESCRIPTION\nMamble seeks for a strong Fron...,Software development,https://staff.am/en/front-end-developer-112,Yerevan,Front-end Developer,100,None,None,"'JavaScript Frameworks', 'JavaScript'",5d15241ac8d3f9226842a407
1,PicsArt,Information technologies,27 July 2019,Permanent,Full time,"'\nAs a QA Engineer at PicsArt, you’ll ensure ...",Quality Assurance /Control,https://staff.am/en/qa-engineer-175,Yerevan,QA Engineer,42,None,None,"'Java', 'Python', 'QA', 'JavaScript'",5d15241ac8d3f9226842a408
2,PicsArt,Information technologies,27 July 2019,Permanent,Full time,'\nPicsArt\xa0is actively seeking for a talent...,Software development,https://staff.am/en/java-engineer,Yerevan,Java Engineer,65,None,None,"'Python', 'Big Data'",5d15241ac8d3f9226842a409
3,Digitain,Information technologies,27 July 2019,Permanent,Full time,"""\nDigitain is looking for a Procurement Speci...",Procurement/Logistics/Courier,https://staff.am/en/procurement-specialist-1,Yerevan,Procurement Specialist,74,email protected,None,"'Flexible', 'Detail-oriented', 'Multitasking'",5d15241ac8d3f9226842a40a
4,PicsArt,Information technologies,27 July 2019,Permanent,Full time,"'\nAs a DevOps Engineer at PicsArt, you’ll hav...",Software development,https://staff.am/en/devops-engineer-63,Yerevan,DevOps Engineer,33,None,None,"'Python', 'DevOps', 'Big Data'",5d15241ac8d3f9226842a40b


In [34]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7820 entries, 0 to 7819
Data columns (total 15 columns):
Company             7805 non-null object
Company Industry    4542 non-null object
Deadline            7805 non-null object
Employment Term     4849 non-null object
Employment Type     4849 non-null object
Job Content         7804 non-null object
Job Industry        4902 non-null object
Job Link            7820 non-null object
Job Location        4835 non-null object
Job Title           7805 non-null object
Job Views           6470 non-null object
Salary              3628 non-null object
Scrape_Date         899 non-null object
Skills              5482 non-null object
_id                 7820 non-null object
dtypes: object(15)
memory usage: 916.5+ KB


In [35]:
#merged.to_excel("Merged.xlsx", index=False)